In [1]:
# imports 
import pandas as pd
from gensim.models.phrases import Phraser, Phrases
from sklearn.model_selection import train_test_split
import logging
import time
from nltk.stem import SnowballStemmer
import multiprocessing
from gensim.models import Word2Vec
from collections import defaultdict
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
import numpy as np
import zipfile
# keras modeling. 
# i love the simplicity, but not having 12.1 cuda support on my system kinda hurts

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.models import load_model

from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [46]:


gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # GPU/CUDA is available
    print("GPU is available")
    # You can also print the details of the available GPUs
    for gpu in gpus:
        print("GPU Name:", gpu.name)
else:
    # No GPU/CUDA available
    print("GPU is not available")


GPU is not available


I dont have tensorflow/keras compatibility until i downgrade from cuda 12.1

cant be bothered tbh

In [2]:

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

to generate progress reports for our stuff


In [3]:
df = pd.read_csv('preprocessed_dataset.csv')
df.target.value_counts()

Negative    748869
Positive    734073
Name: target, dtype: int64

get the data

In [49]:
# df = df.sample(n=160000)
# df.target.value_counts()

# explanation : was trying on smaller dataset to just get a quick idea
# whether stuff works or not
# now i know it works, so no need to limit myself

In [4]:
print(df.isnull().sum())
df.dropna(inplace=True) # clean na

target    0
Clean     1
dtype: int64


In [5]:
# simple train test split

train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
print(train_data.target.value_counts())
print(test_data.target.value_counts())


Negative    524643
Positive    513415
Name: target, dtype: int64
Negative    224225
Positive    220658
Name: target, dtype: int64


In [52]:
train_data.head()

,target,Clean
1199674,Positive,beethoven western x49
855734,Positive,final sleep silenc
517948,Negative,sorri class call what plan get togeth everyon ...
648172,Negative,dad btard mom
801393,Positive,hockey kill tonigh saw gab


Below we start the phrase generation  
saves time on modeling  
a simple bigram phraser

In [6]:

unigrams = [_.split() for _ in df.Clean]
unigrams[0:5]

[['upset',
  'cant',
  'updat',
  'facebook',
  'text',
  'might',
  'cri',
  'result',
  'school',
  'today',
  'also',
  'blah'],
 ['dive',
  'mani',
  'time',
  'ball',
  'manag',
  'save',
  '50',
  'rest',
  'go',
  'bound'],
 ['whole', 'bodi', 'feel', 'itchi', 'like', 'fire'],
 ['behav', 'im', 'mad', 'cant', 'see'],
 ['whole', 'crew']]

In [7]:
phrases = Phrases(unigrams, min_count=40, progress_per=30000)
bigrams = Phraser(phrases)
sentences = bigrams[unigrams]
# sentences[0:5]

2023-06-27 18:10:50,747 : INFO : collecting all words and their counts
2023-06-27 18:10:50,748 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2023-06-27 18:10:50,866 : INFO : PROGRESS: at sentence #10000, processed 76095 words and 65490 word types
2023-06-27 18:10:51,006 : INFO : PROGRESS: at sentence #20000, processed 152132 words and 118301 word types
2023-06-27 18:10:51,129 : INFO : PROGRESS: at sentence #30000, processed 228323 words and 166189 word types
2023-06-27 18:10:51,243 : INFO : PROGRESS: at sentence #40000, processed 305846 words and 212760 word types
2023-06-27 18:10:51,358 : INFO : PROGRESS: at sentence #50000, processed 384509 words and 259058 word types
2023-06-27 18:10:51,456 : INFO : PROGRESS: at sentence #60000, processed 461796 words and 302501 word types
2023-06-27 18:10:51,565 : INFO : PROGRESS: at sentence #70000, processed 539336 words and 343479 word types
2023-06-27 18:10:51,682 : INFO : PROGRESS: at sentence #80000, processed 616824 w

In [8]:
# sanity check time ! im not schizo
# get the number of words in the vocabulary

wrdfreq = defaultdict(int) 
for myphrase in sentences:
    for oneword in myphrase:
        wrdfreq[oneword] += 1
len(wrdfreq) 

454515

In [9]:
# check the most frequent words
# we removed stopwords, so not seen 'the' is expected
sorted(wrdfreq, key=wrdfreq.get, reverse=True)[:10]

['im', 'go', 'get', 'day', 'work', 'love', 'good', 'like', 'today', 'time']

In [10]:
# for workers in building genshimmodel


corecount = multiprocessing.cpu_count()
corecount

16

In [11]:
genshinmodel = Word2Vec(
    min_count= 30, # ignores words which appear less than 30 times in the corpora
    window= 7, # context window size
    vector_size= 300, # size of the vector
    sample= 5e-5, # random downsampling of high freq words
    alpha= 0.04, # learning rate
    min_alpha= 0.005, # minimum rate of learning, where it will stop dropping
    negative= 10, # negative sampling for drowning
    workers= corecount-2
)

2023-06-27 18:12:56,156 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.04>', 'datetime': '2023-06-27T18:12:56.156268', 'gensim': '4.3.0', 'python': '3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [12]:

t = time.time()

# monitor time and build the vocabulary 
# mesh them up real good
genshinmodel.build_vocab(sentences, progress_per=30000)
print("\n\ntook time", (time.time()-t), "s")

2023-06-27 18:13:06,481 : INFO : collecting all words and their counts
2023-06-27 18:13:06,481 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-06-27 18:13:06,721 : INFO : PROGRESS: at sentence #30000, processed 218763 words, keeping 25576 word types
2023-06-27 18:13:06,966 : INFO : PROGRESS: at sentence #60000, processed 442745 words, keeping 40768 word types
2023-06-27 18:13:07,236 : INFO : PROGRESS: at sentence #90000, processed 666765 words, keeping 53370 word types
2023-06-27 18:13:07,498 : INFO : PROGRESS: at sentence #120000, processed 892270 words, keeping 65389 word types
2023-06-27 18:13:07,778 : INFO : PROGRESS: at sentence #150000, processed 1120229 words, keeping 76654 word types
2023-06-27 18:13:08,035 : INFO : PROGRESS: at sentence #180000, processed 1345897 words, keeping 87032 word types
2023-06-27 18:13:08,312 : INFO : PROGRESS: at sentence #210000, processed 1572142 words, keeping 97157 word types
2023-06-27 18:13:08,569 : INFO : PROGRE



took time 13.461478471755981 s


In [60]:
t= time.time()
# and now, train it. the important part
# as expected, takes a bit of time. 
# more epochs will result in better results as a rule of thumb
# but 30 is fine.
# total examples 
genshinmodel.train(
    sentences, # the corpus
    total_examples=genshinmodel.corpus_count, # number of sentences
    epochs=30, # how many epochs to train for
    report_delay=1 # progress report how often, in seconds. can honestly set it to high 10s
    )
print("\n\ntook time", (time.time()-t), "s")

2023-06-27 11:37:11,006 : INFO : Word2Vec lifecycle event {'msg': 'training model with 14 workers on 15590 vocabulary and 300 features, using sg=0 hs=0 sample=5e-05 negative=10 window=7 shrink_windows=True', 'datetime': '2023-06-27T11:37:11.006274', 'gensim': '4.3.0', 'python': '3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2023-06-27 11:37:12,055 : INFO : EPOCH 0 - PROGRESS: at 5.11% examples, 240938 words/s, in_qsize 23, out_qsize 1
2023-06-27 11:37:13,065 : INFO : EPOCH 0 - PROGRESS: at 11.90% examples, 286875 words/s, in_qsize 28, out_qsize 2
2023-06-27 11:37:14,805 : INFO : EPOCH 0 - PROGRESS: at 15.66% examples, 203855 words/s, in_qsize 26, out_qsize 1
2023-06-27 11:37:15,828 : INFO : EPOCH 0 - PROGRESS: at 22.87% examples, 235432 words/s, in_qsize 25, out_qsize 2
2023-06-27 11:37:16,832 : INFO : EPOCH 0 - PROGRESS: at 29.93% examples, 254644 words/s, in_qsize 24, out_qs



took time 532.1568157672882 s


In [13]:

stemmer = SnowballStemmer('english') # stemmer for similarity 

In [62]:
genshinmodel.wv.most_similar(stemmer.stem('test')) # show the similarity checker

[('exam', 0.5778243541717529),
 ('studi', 0.532958984375),
 ('math', 0.5091339945793152),
 ('fail', 0.47623127698898315),
 ('class', 0.4673853814601898),
 ('math_test', 0.46504122018814087),
 ('math_gcse', 0.4571109116077423),
 ('oral_exam', 0.4550704061985016),
 ('calc', 0.4524213969707489),
 ('scienc', 0.45091551542282104)]

In [14]:
# tokenize the inputs
# initiliaze ont he dataset


tknizer = Tokenizer()
tknizer.fit_on_texts(df.Clean)

In [15]:
# ensure they are of the same length by padding. check pad_sequences docu for various params
# doing simple pre-padding here


x_train = pad_sequences(tknizer.texts_to_sequences(train_data.Clean), maxlen=140)
x_test = pad_sequences(tknizer.texts_to_sequences(test_data.Clean), maxlen=140) # twitter absolute max size

print("x_train", x_train.shape)
print("x_test", x_test.shape)

x_train (1038058, 140)
x_test (444883, 140)


In [16]:
# encode labels and transform 

labels = df.target.unique().tolist()
print(labels)

encoder = LabelEncoder()
encoder.fit(df.target.tolist()) # initialize encoder logic

# the usual
# x -> features
# y -> labels
# we encode the labels 

y_train = encoder.transform(train_data.target.tolist()) # encode the target into y_train
y_test = encoder.transform(test_data.target.tolist()) # encode to y_test

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train",y_train.shape)
print("y_test",y_test.shape)

['Negative', 'Positive']
y_train (1038058, 1)
y_test (444883, 1)


In [17]:
# sanity check
# did we reshape, did we matrixizce the inputs
print(
y_test[0:5],
x_test[0])

[[1]
 [1]
 [0]
 [1]
 [1]] [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 1279   44   91  278  283  100   42   52]


In [104]:

# embed the precious genshim word2vec model

embedding_matrix = np.zeros((len(wrdfreq), 300))
for word, i in tknizer.word_index.items():
  if word in genshinmodel.wv:
    embedding_matrix[i] = genshinmodel.wv[word]

# the embedding matrix maps index to vectors

print(embedding_matrix.shape)

(454515, 300)




Time to define model layers  
lets start with our emdedding

In [107]:
embedding_layer = Embedding(len(wrdfreq), 300, weights=[embedding_matrix], input_length=140, trainable=False)

In [108]:
# rest of the model layers
model = Sequential()
model.add(embedding_layer) # highest parameters, most important
model.add(Dropout(0.2))
model.add(LSTM(200, dropout=0.2, return_sequences=True)) # multiple LSTM layers 
# dropouts prevent overfitting, and lower reliance on regularly occuring words
# encourages to actually learn and not take shortcuts
# afterall, its not a real human, we cant have it take shortcuts
model.add(LSTM(100, recurrent_dropout=0.1, return_sequences=True)) 
# recurrent dropout drops the connection between recurernt cells
# i.e; memory and hidden. captures longer term dependancies
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid')) # sigmoid because we need binary output

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 140, 300)          136354500 
                                                                 
 dropout_3 (Dropout)         (None, 140, 300)          0         
                                                                 
 lstm_7 (LSTM)               (None, 140, 200)          400800    
                                                                 
 lstm_8 (LSTM)               (None, 140, 100)          120400    
                                                                 
 lstm_9 (LSTM)               (None, 50)                30200     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 136,905,951
Trainable params: 551,451
No

In [109]:
# compile the model

model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [110]:
# callback so we dont train forever
# model checkpoint added retrospectively
# now im stuck waiting for this to be finished, which will take forever. Sadge


callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=3, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=2e-3, patience=3),
               ModelCheckpoint(filepath='model_checkpoint.h5', save_best_only=True, save_weights_only=False)
               ]

In [ ]:
# add logic to check for model checkpoint to resume training from
# and alternatively also CHECK for models, so that 
# i dont have to train anyways, if one exists. 
# i'll just raise an error for now.
# crude, but works. thats what countsp


In [111]:
t = time.time()
# train. my system takes ~ 3 hrs or so
history = model.fit(x_train, y_train,
                    batch_size=512,
                    epochs=3,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

print("time taken", time.time() - t, "s")

Epoch 1/3
1522/1825 [========================>.....] - ETA: 41:43 - loss: 0.4908 - accuracy: 0.7606

KeyboardInterrupt: 

In [ ]:
# simple evaluation
score = model.evaluate(x_test, y_test, batch_size=512)
print("accuracy:",score[1])
print("loss:",score[0])

In [ ]:

# lets save the model because we dont want to train it for 2 hours every time we open the notebook
model.save('snetiment_analzyer.h5')
# lets zip it too, so we can save space
with zipfile.ZipFile('snetiment_analzyer.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('snetiment_analzyer.h5')

In [ ]:
# since we have a history object from earlier, we can plot how our model evolved
# really helpful to see some cool outputs


acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc)) # 3 for us, so not real 'graph', just a few points

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.show()

Lets gor fo prediction now

In [20]:
# prediction code dummy
# please don't forget to PREDICT, sincerely
# myself
# load the checkpoint
checkpoint = load_model('model_checkpoint.h5')
score = checkpoint.evaluate(x_test, y_test, batch_size=512)
print("accuracy:",score[1])
print("loss:",score[0])

107/869 [==>...........................] - ETA: 11:22 - loss: 0.4503 - accuracy: 0.7881

KeyboardInterrupt: 